<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#建立" data-toc-modified-id="建立-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>建立</a></span></li><li><span><a href="#预处理和建立模型" data-toc-modified-id="预处理和建立模型-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>预处理和建立模型</a></span></li><li><span><a href="#训练" data-toc-modified-id="训练-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>训练</a></span></li></ul></li></ul></div>

_字符型语言模型_

In [1]:
%matplotlib inline
import imp
import utils
from utils import *
from __future__ import division, print_function

C:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras.layers import TimeDistributed,Activation
from numpy.random import choice

## 建立

We haven't really looked into the detail of how this works yet - so this is provided for self-study for those who are interested. We'll look at it closely next week.

In [4]:
path = get_file('nietzsche.txt',origin="https://s3.amazonaws.com/text-datasets/nietzsche.txt") # 尼采的诗歌
text = open(path).read().lower()
print("语料库长度：",len(text))

606208/600901 [==============================] - 7s 11us/step
语料库长度： 600893


In [101]:
#path = 'data/wiki/'
#text = open(path+'small.txt').read().lower()
#print('corpus length:', len(text))

#text = text[0:1000000]

corpus length: 137587200


In [13]:
# 字符型语言模型的字典,只包含不重复的字符，具体见吴恩达课程
chars = sorted(list(set(text)))
vocab_size = len(chars) + 1
print('total chars:',vocab_size)

total chars: 59


In [15]:
chars.insert(0,"\0")

In [17]:
''.join(chars[1:-6])

'\n !"\'(),-.0123456789:;=?[]_abcdefghijklmnopqrstuvwxy'

In [21]:
char_indices = dict((c,i) for i,c in enumerate(chars)) # 字符到索引

In [22]:
indices_char = dict((i, c) for i, c in enumerate(chars)) # 索引 字符 字典

In [28]:
idx = [char_indices[c] for c in text] # 改预料转成数字索引数组

In [29]:
idx[:10]

[43, 45, 32, 33, 28, 30, 32, 1, 1, 1]

In [31]:
# 再把数字索引转成字符

''.join(indices_char[i] for i in idx[:70])

'preface\n\n\nsupposing that truth is a woman--what then? is there not gro'

## 预处理和建立模型

In [32]:
maxLen = 40 # 最大长度
sentences = [] # 生成的句子
next_chars = [] # 下一个字符

In [35]:
len(idx)-maxLen+1

600854

In [36]:
for i in range(0,len(idx)-maxLen+1):
    sentences.append(idx[i:i+maxLen])
    next_chars.append(idx[i+1:i+maxLen+1]) # 下一个字符
print('nb sequences:', len(sentences))

nb sequences: 600854


In [44]:
sentences = np.concatenate([[np.array(o)] for o in sentences[:-2]]) # 拼接起来转化成np格式
next_chars = np.concatenate([[np.array(o)] for o in next_chars[:-2]])

In [46]:
sentences.shape,next_chars.shape

((600852, 40), (600852, 40))

In [48]:
n_fac = 24 # 嵌入矩阵输出大小

In [62]:
# return_sequences: 布尔值。是返回输出序列中的最后一个输出，还是全部序列
# dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于输入的线性转换。
# recurrent_dropout: 在 0 和 1 之间的浮点数。 单元的丢弃比例，用于循环层状态的线性转换。
# implementation: 实现模式，1 或 2。 模式 1 将把它的操作结构化为更多的小的点积和加法操作， 
# 而模式 2 将把它们分批到更少，更大的操作中。 这些模式在不同的硬件和不同的应用中具有不同的性能配置文件。
model = Sequential([
    Embedding(input_dim=vocab_size,output_dim=n_fac,input_length=maxLen),
    LSTM(units=512,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,implementation=2,input_shape=[None,n_fac]),
    Dropout(0.2),
    LSTM(units=512,return_sequences=True,dropout=0.2,recurrent_dropout=0.2,implementation=2),
    Dropout(0.2),
    TimeDistributed(Dense(vocab_size)),
    Activation('softmax')
])

model.compile(loss='sparse_categorical_crossentropy',optimizer=Adam())

In [50]:
# model=Sequential([
#         Embedding(vocab_size, n_fac, input_length=maxLen),
#         LSTM(512, input_dim=n_fac,return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         LSTM(512, return_sequences=True, dropout_U=0.2, dropout_W=0.2,
#              consume_less='gpu'),
#         Dropout(0.2),
#         TimeDistributed(Dense(vocab_size)),
#         Activation('softmax')
#     ])    

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `input_dim` and `input_length` arguments in recurrent layers are deprecated. Use `input_shape` instead.
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, input_shape=(None, 24), dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(512, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, implementation=2)`
  import sys


## 训练

In [ ]:
np.expand_dims()

In [67]:
next_chars.shape

(600852, 40)

In [66]:
np.expand_dims(next_chars,-1).shape

(600852, 40, 1)

In [68]:
sentences.shape

(600852, 40)

In [69]:
# 没看懂
def print_example():
    seed_string="ethics is a basic foundation of all that"
    for i in range(320):
        x=np.array([char_indices[c] for c in seed_string[-40:]])[np.newaxis,:]
        preds = model.predict(x, verbose=0)[0][-1]
        preds = preds/np.sum(preds)
        next_char = choice(chars, p=preds)
        seed_string = seed_string + next_char
    print(seed_string)

In [70]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64,epochs=1)

Epoch 1/1
 47872/600852 [=>............................] - ETA: 2:56:09 - loss: 2.4538

KeyboardInterrupt: 

In [220]:
print_example()

ethics is a basic foundation of all thatscrriets sdi  ,s lrrbmh
fceelsora tec
 n yiefma
cnostencnrut -  o
pen.htt" oaiosovo  stialpts es rb b
ea ie
ohatnmauyielueysiutlmo,es  etfrne oh
ohnio iis e.eosme o rdorfdbteirnse ohdnotafi enicron e eietnyn sytt e ptsrdrede httmi ah
oo, tdye es r,igyct toehitu abrh ei isiem-r natra lnspamlltefae a
cni vuui
twgt fatieh


In [236]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.5152   


In [222]:
print_example()

ethics is a basic foundation of all that he maluces indofely and is; pticrast', and re onerog je ivesantamale as whered
and ror and kytinf? on chaninn nurdeln--ans prory. heke the pepadinar; anf bom,
puntely"" ones to bucf, alcherstol the qisleves: the the wite dadong the gur is prang not galcaula rewinl
more by than sic appads not pepow o mee, a more
bins c


In [235]:
model.optimizer.lr=0.001

In [236]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.5152   


In [237]:
print_example()

ethics is a basic foundation of all that schools pedhaps a new prisons of the ashamed in which
a coverbine estimates of the assumption that one avoid; he will curse about pain:
     people, he-equally present to
the lalier,
nature. that he has
rendered and henceforth distrain and impulses to perceive that each other
former and dangerous, and cannot at
the pu


In [250]:
model.optimizer.lr=0.0001

In [239]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 639s - loss: 1.2892   


In [240]:
print_example()

ethics is a basic foundation of all that account has its granitify them.

131. the new "dilence," out of the
same light,
interpretation thereof: under the "thinking"
there, to counter-arguments in the monality, so many language:
though
all nobilitys of higher impulses, man and hence to everything of seldom man.



chapter i. woman decides according the injur


In [242]:
model.save_weights('data/char_rnn.h5')

In [257]:
model.optimizer.lr=0.00001

In [243]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.2544   


In [249]:
print_example()

ethics is a basic foundation of all that is military
contemplation of distance itself is in physician!

249.. in every
to strick in the man of disguise and in the
will to wind at any progress, the
religious estimates of vehapance has a powerful and religious nature of manner, who had the problem of
decided expression of his equality, which, sometimes power? 


In [258]:
model.fit(sentences, np.expand_dims(next_chars,-1), batch_size=64, nb_epoch=1)

Epoch 1/1
600860/600860 [==============================] - 640s - loss: 1.2218   


In [264]:
print_example()

ethics is a basic foundation of all that
the belief in the importance. the employs concerning
seriousness and
materialism, it is circles which alone is already attained, that he sees
also the day after thinking
of mankind, brightness, resistance--and after the value of "nature" in order to nevertheless
have taken a system of liberal fatalists are willing him


In [283]:
print_example()

ethics is a basic foundation of all that were beought by the temptation of truth--for the rest of a sublime medely and take part of life, which lacks himself the
credibility about this, in short, and raise such a
gods; and on the
other hand, the explanation of
the case, as the most ingredient, and insight, and approach as to the
peculiarly prolonged "distrus


In [282]:
model.save_weights('data/char_rnn.h5')